In [0]:
import warnings
warnings.simplefilter("ignore")

In [0]:
import pandas as pd
import numpy as np
import networkx as nx 
import matplotlib.pyplot as plt

In [0]:
from nilearn import plotting
from nilearn.image import concat_imgs
from nilearn.input_data import NiftiLabelsMasker
from nilearn.image import high_variance_confounds
from nilearn.connectome import ConnectivityMeasure

In [0]:
from nilearn import datasets
dataset = datasets.fetch_atlas_aal(version='SPM12', data_dir=None, url=None, resume=True, verbose=1)
atlas_filename = dataset.maps
labels = dataset.labels
l = len(labels)

In [0]:
masker = NiftiLabelsMasker(labels_img=atlas_filename, background_label = labels, standardize=True, detrend = True, resampling_target = 'labels',
                           low_pass=0.08, high_pass=0.009, t_r=3.7, memory='nilearn_cache', memory_level=1, verbose=0)

In [0]:
correlation_measure = ConnectivityMeasure(kind='correlation')

In [0]:
E = ['001', '003', '006', '009', '010', '011', '024', '026', '031', '032', '034', '039', '045', '050', '056', '059', '068', '072','078', '079', '081', '083', '084', '092', '094']

In [0]:
DE = ['002', '015', '017', '022', '025', '027', '038', '040', '044', '048', '049', '055', '057', '061', '064', '073', '074', '075', '076', '086', '087', '088', '096', '099', '105']

In [0]:
C = ['008', '013', '018', '033', '036', '037', '043', '046', '047', '051', '054', '062', '077', '080', '082', '085', '089', '090', '091', '093', '098', '100', '106', '107', '108']

In [0]:
D = ['004','012','014','016', '019', '020','021','023', '028', '029', '030', '035', '041', '042', '052', '053', '058', '060', '063', '065', '066', '067', '069', '070', '071']

In [0]:
P = E+DE+C+D
len(P)

100

In [0]:
P = np.sort(P)

In [0]:
for i in range(len(P)):
    img = concat_imgs("Brain_functional_data/P"+P[i]+"/s"+P[i]+"*.nii", auto_resample=True, verbose=0)
    confounds = high_variance_confounds(img, l)
    time_series = masker.fit_transform(img, confounds)
    correlation_matrix = correlation_measure.fit_transform([time_series])[0]
    np.fill_diagonal(correlation_matrix, 0)
    Data = pd.DataFrame(correlation_matrix)
    Data.to_csv('functional_connenctome/correlation_matrix/P_'+P[i]+'.csv', sep=',')

In [0]:
labels.append('0')

In [0]:
labels.append('1')

In [0]:
column_labels = []

In [0]:
degree_label = ['degree_'+ labels[i] for i in range(len(labels))]
clustering_coefficient_label = ['clustering_coefficient_'+ labels[i] for i in range(len(labels))]
average_neighbor_degree_label = ['average_neighbor_degree_'+ labels[i] for i in range(len(labels))]
degree_centrality_label = ['degree_centrality_'+ labels[i] for i in range(len(labels))]
closeness_centrality_label = ['closeness_centrality_'+ labels[i] for i in range(len(labels))]
betweenness_centrality_label = ['betweenness_centrality_'+ labels[i] for i in range(len(labels))]
eigenvector_centrality_label = ['eigenvector_centrality_'+ labels[i] for i in range(len(labels))]
shortest_path_length_label = ['path_length_'+ labels[i] for i in range(len(labels))]
average_clustering_label = ['average_clustering']
local_efficiency_label = ['local_efficiency']
global_efficiency_label = ['global_efficiency']
index_label = ['patient']
target_label = ['target']

In [0]:
column_labels = degree_label + clustering_coefficient_label + average_neighbor_degree_label + degree_centrality_label + closeness_centrality_label + betweenness_centrality_label + eigenvector_centrality_label + shortest_path_length_label + average_clustering_label + local_efficiency_label + global_efficiency_label + index_label +  target_label 

In [0]:
len(column_labels)

941

In [0]:
treashold = np.arange(0.5,0.7,0.1)
treashold

array([ 0.5,  0.6])

In [0]:
for tr in treashold:
    Data = pd.DataFrame(columns = column_labels)
    for i in range(len(P)):
        ft = []
        correlation_matrix = pd.read_csv('functional_connenctome/correlation_matrix/P_'+P[i]+'.csv', sep=',').drop('Unnamed: 0', axis = 1)
        binary_matrix = abs(correlation_matrix) > tr
        G = nx.from_pandas_adjacency(binary_matrix)
        degree = [G.degree(i) for i in G.nodes()]
        clustering_coefficient = nx.clustering(G)
        average_neighbor_degree = nx.average_neighbor_degree(G)
        degree_centrality = nx.degree_centrality(G)
        closeness_centrality = nx.closeness_centrality(G)
        betweenness_centrality = nx.betweenness_centrality(G)
        eigenvector_centrality = nx.eigenvector_centrality(G)
        shortest_path_length = []
         for i in G.nodes():
            shortest_path_length.append(sum(nx.single_source_shortest_path_length(G, i).values()))
        average_clustering = nx.average_clustering(G)
        local_efficiency = nx.local_efficiency(G)
        global_efficiency = nx.global_efficiency(G)
        ft.extend(degree)
        ft.extend(list(clustering_coefficient.values()))
        ft.extend(list(average_neighbor_degree.values()))
        ft.extend(list(degree_centrality.values()))
        ft.extend(list(closeness_centrality.values()))
        ft.extend(list(betweenness_centrality.values()))
        ft.extend(list(eigenvector_centrality.values()))
        ft.extend(shortest_path_length)
        ft.append(average_clustering)
        ft.append(local_efficiency)
        ft.append(global_efficiency)
        ft.append('P'+P[i]) 
        if P[i] in E:
            ft.append('E')
        if P[i] in D:
            ft.append('D')
        if P[i] in DE:
            ft.append('DE')
        if P[i] in C:
            ft.append('C')
        Data.loc[i] = ft    
    Data = Data.rename(columns={587: "index", 588: "target"})
    Data = Data.set_index('index')
    Data.to_csv('fMRI_features\partial_correlation_matrix/features_'+str(tr)+'.csv', sep=',')

### t test без корректировки

In [0]:
import math
from scipy.stats import t

In [0]:
correlation_matrix = pd.read_csv('clean_last/P'+P[i]+'.csv', sep=',').drop('Unnamed: 0', axis = 1)

In [0]:
Data = pd.DataFrame(columns = column_labels)
for i in range(len(P)):
    ft = []
    correlation_matrix = pd.read_csv('clean_last/P'+P[i]+'.csv', sep=',').drop('Unnamed: 0', axis = 1)
    correlation_matrix = correlation_matrix.as_matrix()
    np.fill_diagonal(correlation_matrix, 0)
    corr_list = np.reshape(correlation_matrix, (len(correlation_matrix)*len(correlation_matrix)))
    transform = list(map(lambda x: x/math.sqrt(1-x**2)*math.sqrt(len(correlation_matrix)-2), corr_list))
    t_krit = np.abs(t.ppf(0.01/2, len(correlation_matrix)-2, loc=0, scale=1))
    bin_list = np.abs(transform) > t_krit
    binary_matrix = np.reshape(bin_list, (len(correlation_matrix), len(correlation_matrix)))
    G = nx.from_numpy_matrix(binary_matrix)
    degree = [G.degree(i) for i in G.nodes()]
    clustering_coefficient = nx.clustering(G)
    average_neighbor_degree = nx.average_neighbor_degree(G)
    degree_centrality = nx.degree_centrality(G)
    closeness_centrality = nx.closeness_centrality(G)
    betweenness_centrality = nx.betweenness_centrality(G)
    eigenvector_centrality = nx.eigenvector_centrality(G)
    shortest_path_length = []
    for k in G.nodes():
        shortest_path_length.append(sum(nx.single_source_shortest_path_length(G, k).values()))
    average_clustering = nx.average_clustering(G)
    local_efficiency = nx.local_efficiency(G)
    global_efficiency = nx.global_efficiency(G)
    ft.extend(degree)
    ft.extend(list(clustering_coefficient.values()))
    ft.extend(list(average_neighbor_degree.values()))
    ft.extend(list(degree_centrality.values()))
    ft.extend(list(closeness_centrality.values()))
    ft.extend(list(betweenness_centrality.values()))
    ft.extend(list(eigenvector_centrality.values()))
    ft.extend(shortest_path_length)
    ft.append(average_clustering)
    ft.append(local_efficiency)
    ft.append(global_efficiency)
    ft.append('P'+P[i]) 
    if P[i] in E:
        ft.append('E')
    if P[i] in D:
        ft.append('D')
    if P[i] in DE:
        ft.append('DE')
    if P[i] in C:
        ft.append('C')
    Data.loc[i] = ft    
Data.to_csv('fmri_feature_correlation_significance/0.01/manual/features_ttest.csv', sep=',')

In [0]:
for i in range(len(P)):
    correlation_matrix = pd.read_csv('functional_connenctome/correlation_matrix/P_'+P[i]+'.csv', sep=',').drop('Unnamed: 0', axis = 1)
    correlation_matrix = correlation_matrix.as_matrix()
    np.fill_diagonal(correlation_matrix, 0)
    corr_list = np.reshape(correlation_matrix, (len(correlation_matrix)*len(correlation_matrix)))
    transform = list(map(lambda x: x/math.sqrt(1-x**2)*math.sqrt(len(correlation_matrix)-2), corr_list))
    t_krit = np.abs(t.ppf(0.1/2, len(correlation_matrix)-2, loc=0, scale=1))
    corr_list_ttest = [0]*(len(correlation_matrix)*len(correlation_matrix))
    for j in range(len(transform)):
        if np.abs(transform[j]) > t_krit:
            corr_list_ttest[j] = corr_list[j]
    correlation_matrix_ttest = np.reshape(corr_list_ttest, (len(correlation_matrix), len(correlation_matrix))) 
    correlation_matrix_ttest = pd.DataFrame(correlation_matrix_ttest)
    correlation_matrix_ttest.to_csv('functional_connenctome/correlation_matrix_ttest/0.1/P_'+P[i]+'.csv', sep=',')

#### Bonferoni

In [0]:
Data = pd.DataFrame(columns = column_labels)
for i in range(len(P)):
    ft = []
    correlation_matrix = pd.read_csv('clean_last/P'+P[i]+'.csv', sep=',').drop('Unnamed: 0', axis = 1)
    correlation_matrix = correlation_matrix.as_matrix()
    np.fill_diagonal(correlation_matrix, 0)
    corr_list = np.reshape(correlation_matrix, (len(correlation_matrix)*len(correlation_matrix)))
    transform = list(map(lambda x: x/math.sqrt(1-x**2)*math.sqrt(len(correlation_matrix)-2), corr_list))
    t_krit = np.abs(t.ppf((0.1/2)/(len(correlation_matrix)*len(correlation_matrix)), len(correlation_matrix)-2))
    bin_list = np.abs(transform) > t_krit
    binary_matrix = np.reshape(bin_list, (len(correlation_matrix), len(correlation_matrix)))
    G = nx.from_numpy_matrix(binary_matrix)
    degree = [G.degree(i) for i in G.nodes()]
    clustering_coefficient = nx.clustering(G)
    average_neighbor_degree = nx.average_neighbor_degree(G)
    degree_centrality = nx.degree_centrality(G)
    closeness_centrality = nx.closeness_centrality(G)
    betweenness_centrality = nx.betweenness_centrality(G)
    #eigenvector_centrality = nx.eigenvector_centrality(G)
    shortest_path_length = []
    for k in G.nodes():
        shortest_path_length.append(sum(nx.single_source_shortest_path_length(G, k).values()))
    average_clustering = nx.average_clustering(G)
    local_efficiency = nx.local_efficiency(G)
    global_efficiency = nx.global_efficiency(G)
    ft.extend(degree)
    ft.extend(list(clustering_coefficient.values()))
    ft.extend(list(average_neighbor_degree.values()))
    ft.extend(list(degree_centrality.values()))
    ft.extend(list(closeness_centrality.values()))
    ft.extend(list(betweenness_centrality.values()))
    #ft.extend(list(eigenvector_centrality.values()))
    ft.extend(shortest_path_length)
    ft.append(average_clustering)
    ft.append(local_efficiency)
    ft.append(global_efficiency)
    ft.append('P'+P[i]) 
    if P[i] in E:
        ft.append('E')
    if P[i] in D:
        ft.append('D')
    if P[i] in DE:
        ft.append('DE')
    if P[i] in C:
        ft.append('C')
    Data.loc[i] = ft    
Data.to_csv('fmri_feature_correlation_significance/0.1/manual/features_bonferoni.csv', sep=',')

#### Benjamini–Hochberg–Yekutieli procedure

In [0]:
def c(x):
    sum = 0
    for i in range(1,x+1):
        sum = sum + 1./i
    return sum

In [0]:
Data = pd.DataFrame(columns = column_labels)
for i in range(len(P)):
    ft = []
    correlation_matrix = pd.read_csv('clean_last/P'+P[i]+'.csv', sep=',').drop('Unnamed: 0', axis = 1)
    correlation_matrix = correlation_matrix.as_matrix()
    np.fill_diagonal(correlation_matrix, 0)
    corr_list = np.reshape(correlation_matrix, (len(correlation_matrix)*len(correlation_matrix)))
    transform = list(map(lambda x: x/math.sqrt(1-x**2)*math.sqrt(len(correlation_matrix)-2), corr_list))
    pvalue = np.abs(t.sf(transform, len(correlation_matrix)-2))
    ind = np.argsort(pvalue)
    pvalue_sort = np.sort(pvalue)
    bin_list = [False]*(len(correlation_matrix)*len(correlation_matrix))
    cm = c(len(correlation_matrix)*len(correlation_matrix))
    for j in range(len(bin_list)):
        if 2*pvalue_sort[j]*(len(correlation_matrix)*len(correlation_matrix))*cm/(j+1) < 0.05:
            bin_list[j] = True
        else: break
    bin_list_order = [False]*(len(correlation_matrix)*len(correlation_matrix)) 
    for j in range(len(bin_list)):
        bin_list_order[ind[j]] = bin_list[j]
    binary_matrix = np.reshape(bin_list_order, (len(correlation_matrix), len(correlation_matrix))) 
    G = nx.from_numpy_matrix(binary_matrix)
    degree = [G.degree(x) for x in G.nodes()]
    clustering_coefficient = nx.clustering(G)
    average_neighbor_degree = nx.average_neighbor_degree(G)
    degree_centrality = nx.degree_centrality(G)
    closeness_centrality = nx.closeness_centrality(G)
    betweenness_centrality = nx.betweenness_centrality(G)
    eigenvector_centrality = nx.eigenvector_centrality(G)
    shortest_path_length = []
    for k in G.nodes():
        shortest_path_length.append(sum(nx.single_source_shortest_path_length(G, k).values()))
    average_clustering = nx.average_clustering(G)
    local_efficiency = nx.local_efficiency(G)
    global_efficiency = nx.global_efficiency(G)
    ft.extend(degree)
    ft.extend(list(clustering_coefficient.values()))
    ft.extend(list(average_neighbor_degree.values()))
    ft.extend(list(degree_centrality.values()))
    ft.extend(list(closeness_centrality.values()))
    ft.extend(list(betweenness_centrality.values()))
    ft.extend(list(eigenvector_centrality.values()))
    ft.extend(shortest_path_length)
    ft.append(average_clustering)
    ft.append(local_efficiency)
    ft.append(global_efficiency)
    ft.append('P'+P[i]) 
    if P[i] in E:
        ft.append('E')
    if P[i] in D:
        ft.append('D')
    if P[i] in DE:
        ft.append('DE')
    if P[i] in C:
        ft.append('C')
    Data.loc[i] = ft    
Data.to_csv('fmri_feature_correlation_significance/0.05/manual/features_BHY.csv', sep=',')